In [ ]:
import numpy as np
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('using device:', device)

In [ ]:
from model import load_dataset

In [ ]:
from model import create_model

n_classes = 251
model = create_model(n_classes, device)